In [1]:
# 导入必要的库
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import torch
import torch.nn as nn
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import torch.nn.functional as F
import nbimporter
from network import *
import os
from utils import *
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import matplotlib

In [2]:
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
num_class=4
num_epoch=2500
test_inverval = 50
num_view = 3
dim_hvcdn= 64
batch_size = 101

In [4]:
# 读取制表符分隔的CSV文件
Methylation_train = pd.read_csv('../../DATA/BLCA_parting/1_tr.csv')
# 显示 DataFrame
Methylation_train

,0.0752463054187192,0.9911882142365416,0.6800423784929148,0.969684935133704,0.9089919103920348,0.046335020911292,0.032830429239269,0.983728813559322,0.9732649498035792,0.108658355255633,...,0.2962497141550423,0.1931479642502482,0.8875406925125778,0.0380033357159876,0.9659176975511234,0.94608195542775,0.9141064474253572,0.6571871040405464,0.527659005879209,0.7971216341689878
0,0.322167,0.739777,0.968878,0.505030,0.769757,0.420977,0.567255,0.902486,0.713771,0.970948,...,0.968100,0.208540,0.802012,0.930665,0.862282,0.787203,0.855257,0.972265,0.301042,0.684308
1,0.786453,0.432741,0.811681,0.432883,0.484287,0.922518,0.952401,0.764520,0.643278,0.671737,...,0.795907,0.439424,0.161586,0.865857,0.574729,0.613515,0.429035,0.924257,0.094602,0.995125
2,0.508005,0.633485,0.970335,0.701747,0.586185,0.950693,0.560561,0.578757,0.709515,0.441153,...,0.537389,0.292949,0.598698,0.661306,0.895228,0.692451,0.655344,0.773617,0.182924,0.955896
3,0.300246,0.628115,0.838564,0.356367,0.791693,0.768105,0.379516,0.580791,0.446639,0.541690,...,0.452321,0.284012,0.560817,0.472719,0.833502,0.448886,0.535482,0.900324,0.412079,0.897981
4,0.614039,0.701776,0.911138,0.509664,0.651836,0.685560,0.665427,0.880565,0.538848,0.517671,...,0.515436,0.298908,0.479728,0.753395,0.692881,0.642847,0.421679,0.807968,0.105024,0.806175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,0.915887,0.607600,0.938021,0.533492,0.298538,0.928021,0.953145,0.865311,0.542012,0.931259,...,0.822319,0.270109,0.491270,0.854658,0.353699,0.697628,0.533535,0.876813,0.190807,0.965297
297,0.222291,0.774611,0.425904,0.544612,0.949751,0.052939,0.046111,0.963955,0.915976,0.119181,...,0.158701,0.249255,0.839006,0.037646,0.972734,0.951689,0.885980,0.364775,1.000000,0.648909
298,0.354557,0.673276,0.944378,0.458830,0.753889,0.979419,0.940395,0.871751,0.616652,0.598307,...,0.401898,0.452830,0.676532,0.519538,0.902045,0.621136,0.601904,0.741236,0.057723,0.977368
299,0.316010,0.719813,0.820554,0.513768,0.901369,0.463790,0.124947,0.791638,0.553252,0.389683,...,0.382689,0.253227,0.566440,0.587801,0.917319,0.472178,0.315664,0.797691,0.127873,0.865367


In [5]:
# 读取制表符分隔的CSV文件
Methylation_test = pd.read_csv('../../DATA/BLCA_parting/1_te.csv')
# 显示 DataFrame
Methylation_test

,0.8200738916256156,0.499793473771169,0.7425506555423123,0.3591474715382579,0.6029869321717485,0.8518600044023772,0.7771993200169997,0.6792090395480226,0.5947184635530336,0.6680773189980557,...,0.7537159844500342,0.2452830188679242,0.1787511097957975,0.7158684774839171,0.3797020954304468,0.4990654205607476,0.1867157074859368,0.8751231873856118,0.14470871191876,0.8487697307335189
0,0.109606,0.888751,0.674877,0.731136,0.734599,0.800242,0.084254,0.517175,0.596683,0.259522,...,0.174365,0.111718,0.730098,0.085895,0.723933,0.737599,0.894202,0.498663,0.159808,0.638812
1,0.233744,0.771720,0.436101,0.864046,0.923460,0.484922,0.270506,0.925424,0.880838,0.288116,...,0.236108,0.406157,0.678899,0.559209,0.944585,0.817541,0.724794,0.861608,0.151390,0.685469
2,0.246429,0.942723,0.978149,0.771909,0.981487,0.203170,0.048874,0.898870,0.857704,0.377788,...,0.255888,0.395233,0.851435,0.918037,0.943449,0.907692,0.857205,0.712657,0.118520,0.749768
3,0.109236,0.934325,0.922262,0.926794,0.923927,0.250275,0.064386,0.507910,0.891205,0.118838,...,0.098902,0.110228,0.693104,0.038122,0.872507,0.839396,0.815015,0.125862,0.107296,0.293756
4,0.820936,0.509018,0.916038,0.506222,0.459241,0.943980,0.778687,0.824181,0.503165,0.613519,...,0.827350,0.254220,0.278485,0.713128,0.674451,0.536161,0.268066,0.915388,0.024586,0.948932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.513177,0.917114,0.685207,0.369473,0.885190,0.115672,0.091585,0.978305,0.849520,0.116322,...,0.378459,0.121648,0.774489,0.097689,0.960237,0.870165,0.839031,0.494580,0.077098,0.339949
96,0.290394,0.330029,0.873659,0.685597,0.906970,0.954876,0.102635,0.963277,0.780554,0.721606,...,0.628630,0.251738,0.708790,0.175959,0.908988,0.838821,0.737343,0.872167,0.079236,0.976091
97,0.401970,0.672587,0.384585,0.302224,0.803049,0.231015,0.147471,0.898305,0.706787,0.909184,...,0.954493,0.361470,0.671796,0.613057,0.895102,0.777426,0.109260,0.725609,0.316542,0.935817
98,0.337562,0.782459,0.877765,0.784088,0.670504,0.945741,0.328836,0.632542,0.650153,0.510122,...,0.476904,0.603774,0.767091,0.552657,0.843221,0.724515,0.803332,0.851471,0.121459,0.957869


In [6]:
# # 读取制表符分隔的CSV文件
# Methylation_val = pd.read_csv('../../DATA/BLCA_parting/methylation_val.csv')
# # 显示 DataFrame
# Methylation_val

In [7]:
# 读取制表符分隔的CSV文件
miRNASeq_train = pd.read_csv('../../DATA/BLCA_parting/3_tr.csv')
# 显示 DataFrame
miRNASeq_train

,0.0532462506047412,0.6213407206570001,0.5117845583556946,0.7412033357304164,0.5261464038699875,0.7201409811835597,0.2506968964876417,0.0,0.1802774662617658,0.4441372631710479,...,0.5514555973918632,0.0.2,0.1591204831336959,0.0694270464571646,0.5643671444733146,0.2063787901732152,0.0.3,0.4564890760856177,0.1791856570896179,0.1370675521861261
0,0.278616,0.439644,0.585549,0.828293,0.591552,0.691661,0.553106,0.000000,0.304608,0.458382,...,0.456387,0.174968,0.450202,0.000000,0.136678,0.498864,0.0,0.521675,0.425050,0.407370
1,0.635172,0.301839,0.422875,0.570733,0.415744,0.335361,0.598196,0.000000,0.482970,0.504069,...,0.473781,0.049656,0.539766,0.831314,0.000000,0.311380,0.0,0.325427,0.502925,0.489051
2,0.527054,0.343373,0.434881,0.533603,0.381781,0.281185,0.352899,0.000000,0.354355,0.495666,...,0.248673,0.046016,0.425144,0.199635,0.056106,0.113924,0.0,0.230732,0.331092,0.353895
3,0.259816,0.214468,0.626241,0.714003,0.621317,0.598924,0.443842,0.000000,0.386852,0.645456,...,0.577519,0.000000,0.349115,0.190989,0.000000,0.317274,0.0,0.366302,0.364675,0.233117
4,0.471224,0.238654,0.324850,0.408501,0.164794,0.310860,0.309226,0.029163,0.198233,0.449948,...,0.211002,0.000000,0.306696,0.702957,0.090742,0.091535,0.0,0.143045,0.335608,0.262001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,0.388592,0.022909,0.000000,0.000000,0.000000,0.000000,0.271804,0.000000,0.142753,0.223482,...,0.000000,0.007443,0.329485,0.596781,0.000000,0.000000,0.0,0.000000,0.298215,0.164040
297,0.234649,0.538371,0.317326,0.545564,0.605125,0.600970,0.215427,0.000000,0.109260,0.292489,...,0.419781,0.044486,0.141314,0.000000,0.000000,0.174252,0.0,0.398926,0.056052,0.132682
298,0.426144,0.394907,0.501950,0.648953,0.349058,0.333731,0.516232,0.000000,0.387546,0.509551,...,0.466930,0.000000,0.483579,0.057159,0.036426,0.315270,0.0,0.458127,0.432461,0.384334
299,0.313875,0.756533,0.866373,0.842701,0.791588,0.393572,0.465566,0.000000,0.376583,0.511707,...,0.911415,0.067112,0.343666,0.000000,0.000000,0.789637,0.0,0.761201,0.348408,0.274939


In [8]:
# 读取制表符分隔的CSV文件
miRNASeq_test = pd.read_csv('../../DATA/BLCA_parting/3_te.csv')
# 显示 DataFrame
miRNASeq_test

,0.4608805031446541,0.1187276513811936,0.2821793133973277,0.3808586604090181,0.3314076680126334,0.3688249673128305,0.3985074188910298,0.0,0.2672975390935094,0.3747247083079518,...,0.3353659656534116,0.0151006882487891,0.2896244147119993,0.8896438102499863,0.0.1,0.1569420014677274,0.0.2,0.2102514840267,0.3648511402647327,0.3075509330325909
0,0.236913,0.683111,0.468740,0.715012,0.594271,0.546833,0.296492,0.000000,0.078804,0.432501,...,0.330554,0.035310,0.260463,0.047152,0.055385,0.154007,0.000000,0.423508,0.242708,0.197634
1,0.225206,0.817851,0.760215,0.861953,0.622496,0.702196,0.424290,0.000000,0.152556,0.611952,...,0.789825,0.041795,0.276456,0.099232,0.123053,0.396286,0.000000,0.524508,0.288806,0.230392
2,0.153033,0.459311,0.739274,0.917805,0.764063,0.864874,0.384843,0.000000,0.248523,0.271293,...,0.653504,0.034678,0.196001,0.000000,0.000000,0.398919,0.000000,0.547464,0.269265,0.225097
3,0.202313,0.638060,0.683836,0.901783,0.567985,0.632937,0.475210,0.000000,0.249153,0.550458,...,0.558711,0.000000,0.219913,0.164084,0.000000,0.266824,0.000000,0.520407,0.322833,0.362854
4,0.476613,0.203819,0.512656,0.468832,0.663995,0.356034,0.541017,0.000000,0.404355,0.576105,...,0.528680,0.000000,0.424255,0.523692,0.086788,0.354596,0.000000,0.335624,0.463954,0.403256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.200726,0.669398,0.507829,0.759434,0.550054,0.692286,0.380207,0.000000,0.242235,0.246786,...,0.602700,0.000000,0.247028,0.210392,0.028830,0.274780,0.000000,0.537563,0.254065,0.262440
96,0.218423,0.319738,0.549891,0.618561,0.373926,0.435469,0.396757,0.000000,0.263782,0.635787,...,0.468803,0.000000,0.284842,0.055539,0.000000,0.204608,0.000000,0.417362,0.335977,0.275555
97,0.421993,0.890565,0.463056,0.684569,0.593711,0.349080,0.540508,0.044321,0.429934,0.784765,...,0.938782,0.084986,0.411431,0.140297,0.311909,0.255466,0.000000,0.585372,0.446990,0.480018
98,0.264586,0.170537,0.517054,0.704894,0.505417,0.619351,0.351275,0.028393,0.246431,0.576387,...,0.404463,0.021830,0.298811,0.029151,0.000000,0.385278,0.000000,0.374701,0.287664,0.210311


In [9]:
# # 读取制表符分隔的CSV文件
# miRNASeq_val = pd.read_csv('../../DATA/BLCA_parting/miRNASeq_val.csv')
# # 显示 DataFrame
# miRNASeq_val

In [10]:
# 读取制表符分隔的CSV文件
RNAseq_train = pd.read_csv('../../DATA/BLCA_parting/2_tr.csv')
# 显示 DataFrame
RNAseq_train

,0.377954420035459,0.3005685724575208,0.2730827836798193,0.1161187365375847,0.4779390373985842,0.4707477391785799,0.2865889005278199,0.0966272431834859,0.1977892200859747,0.2753795102028558,...,0.7088748208313427,0.1294118931429382,0.8746186743734583,0.0973485524047321,0.2968674553355868,0.0.94,0.5380459729529323,0.2629923281866915,0.4297671622465255,0.0.95
0,0.708650,0.767016,0.813089,0.680699,0.545733,0.572325,0.697678,0.842246,0.474116,0.704920,...,0.804933,0.000000,0.843018,0.000000,0.608175,0.662687,0.595088,0.573621,0.679507,0.000000
1,0.836337,0.724553,0.922874,0.349111,0.514206,0.372578,0.746362,0.622469,0.501846,0.734891,...,0.629348,0.000000,0.710397,0.000000,0.592317,0.176802,0.825509,0.166148,0.224444,0.000000
2,0.489098,0.636059,0.714290,0.630977,0.069949,0.370207,0.785841,0.417892,0.811071,0.595715,...,0.615886,0.156727,0.817037,0.192973,0.574707,0.000000,0.862929,0.000000,0.429698,0.000000
3,0.477439,0.393592,0.607329,0.198135,0.550840,0.245304,0.717082,0.389015,0.608288,0.590069,...,0.601839,0.056409,0.753295,0.153588,0.498053,0.138553,0.739054,0.000000,0.218294,0.471895
4,0.758234,0.549616,0.792716,0.529613,0.210826,0.430117,0.766417,0.452830,0.812468,0.704500,...,0.850681,0.000000,0.749091,0.000000,0.553597,0.093133,0.954716,0.413251,0.623621,0.060177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,0.799387,0.731254,0.918431,0.336832,0.343084,0.259899,0.845376,0.465362,0.722849,0.705281,...,0.558242,0.000000,0.803681,0.139736,0.550429,0.390241,0.716650,0.000000,0.168189,0.242301
297,0.409785,0.419107,0.306179,0.119020,0.700423,0.340862,0.337622,0.285221,0.222367,0.235409,...,0.797002,0.137360,0.816745,0.000000,0.116475,0.052727,0.282079,0.304016,0.434091,0.000000
298,0.446395,0.897199,0.701386,0.279609,0.136374,0.446684,0.746707,0.120624,0.777950,0.692773,...,0.543753,0.273617,0.718979,0.000000,0.593554,0.683562,0.887407,0.306006,0.351222,0.000000
299,0.338903,0.581646,0.574051,0.240783,0.494790,0.491801,0.608010,0.733429,0.085286,0.211625,...,0.807453,0.000000,0.816701,0.000000,0.550066,0.099365,0.324379,0.332649,0.577956,0.081455


In [11]:
# 读取制表符分隔的CSV文件
RNAseq_test = pd.read_csv('../../DATA/BLCA_parting/2_te.csv')
# 显示 DataFrame
RNAseq_test

,0.9005624112059554,0.5990745692054172,0.7037518514177332,0.3540163199849064,0.5065347857473791,0.366018824798762,0.5739812262913282,0.5450923958105004,0.1545521416086865,0.7565278757939309,...,0.6094123268036311,0.0.43,0.8143071915458833,0.0.44,0.6996318203766299,0.5053215970544241,0.7397337664630486,0.0.45,0.3709449929478134,0.7210442364951085
0,0.296763,0.614091,0.406726,0.418628,0.221946,0.356436,0.258754,0.156134,0.186783,0.168306,...,0.737183,0.362351,0.455971,0.000000,0.269179,0.000000,0.216800,0.000000,0.690976,0.000000
1,0.298841,0.598496,0.566390,0.633547,0.303725,0.638496,0.271771,0.483022,0.283602,0.290725,...,0.749032,0.145198,0.861862,0.395339,0.661743,0.127527,0.296485,0.548807,0.784157,0.181806
2,0.389895,0.633211,0.440238,0.583054,0.598905,0.756300,0.416368,0.302929,0.087810,0.533251,...,0.712183,0.498553,0.710674,0.000000,0.363774,0.000000,0.516445,0.000000,0.581285,0.000000
3,0.410995,0.435171,0.572187,0.211201,0.419886,0.474694,0.252143,0.108940,0.461106,0.482530,...,0.600239,0.363650,0.854082,0.000000,0.592196,0.065384,0.472759,0.000000,0.537955,0.000000
4,0.605432,0.721814,0.916524,0.462494,0.000000,0.196283,0.779200,0.329356,0.759804,0.819772,...,0.485726,0.000000,0.701771,0.000000,0.727743,0.089269,0.936584,0.000000,0.233254,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.533469,0.499896,0.490959,0.120238,0.445001,0.721120,0.122588,0.316071,0.353617,0.372783,...,0.629025,0.362067,0.815986,0.000000,0.335013,0.053456,0.262332,0.307707,0.415640,0.000000
96,0.409586,0.876933,0.546251,0.187853,0.294599,0.215861,0.847451,0.104004,0.227327,0.739486,...,0.580972,0.171912,0.713535,0.106771,0.430091,0.084954,0.583860,0.115302,0.328724,0.000000
97,0.381935,0.617660,0.563428,0.442086,0.111592,0.364031,0.749830,0.387519,0.557642,0.217976,...,0.721011,0.000000,0.717155,0.128708,0.608040,0.406225,0.486397,0.000000,0.464774,0.987739
98,0.311886,0.847446,0.495349,0.710949,0.172253,0.236928,0.735460,0.471042,0.515579,0.389796,...,0.708743,0.000000,0.843952,0.000000,0.692736,0.337629,0.640858,0.239206,0.344702,0.000000


In [12]:
# # 读取制表符分隔的CSV文件
# RNAseq_val = pd.read_csv('../../DATA/BLCA_parting/RNAseq_val.csv')
# # 显示 DataFrame
# RNAseq_val

In [13]:
# 使用np.load()函数加载.npy文件
labels_train = pd.read_csv('../../DATA/BLCA_parting/labels_tr.csv')
labels_train.value_counts()

1
2    102
1     99
3     98
0      2
Name: count, dtype: int64

In [14]:
# 使用np.load()函数加载.npy文件
labels_test = pd.read_csv('../../DATA/BLCA_parting/labels_te.csv')
labels_test.value_counts()

3
2    36
3    34
1    30
Name: count, dtype: int64

In [15]:
# # 使用np.load()函数加载.npy文件
# labels_val = pd.read_csv('../../DATA/BLCA_parting/labels_val.csv')
# labels_val.value_counts()

In [16]:
label_dim = len(np.unique(labels_train))
label_dim

4

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 使用 GPU 

In [18]:
# 调用函数，将这些 numpy 数组转换为 tensors
train_tensors = batch_dataframe_to_tensors(Methylation_train, miRNASeq_train, RNAseq_train)

# 现在 tensors 是一个包含这些转换后张量的列表
Methylation_train = train_tensors[0]
miRNASeq_train = train_tensors[1]
RNAseq_train = train_tensors[2]

DataFrame 0 已转换为 Tensor，并移动到 cuda:0
DataFrame 1 已转换为 Tensor，并移动到 cuda:0
DataFrame 2 已转换为 Tensor，并移动到 cuda:0


In [19]:
# 调用函数，将这些 numpy 数组转换为 tensors
test_tensors = batch_dataframe_to_tensors(Methylation_test, miRNASeq_test, RNAseq_test)

# 现在 tensors 是一个包含这些转换后张量的列表
Methylation_test = test_tensors[0]
miRNASeq_test = test_tensors[1]
RNAseq_test = test_tensors[2]

DataFrame 0 已转换为 Tensor，并移动到 cuda:0
DataFrame 1 已转换为 Tensor，并移动到 cuda:0
DataFrame 2 已转换为 Tensor，并移动到 cuda:0


In [20]:
# # 调用函数，将这些 numpy 数组转换为 tensors
# val_tensors = batch_dataframe_to_tensors(Methylation_val, miRNASeq_val, RNAseq_val)

# # 现在 tensors 是一个包含这些转换后张量的列表
# Methylation_val = val_tensors[0]
# miRNASeq_val = val_tensors[1]
# RNAseq_val = val_tensors[2]

In [21]:
# 将DataFrame的值转换为Tensor
labels_tr_tensor = torch.LongTensor(labels_train.values)
labels_tr_tensor

tensor([[3],
        [1],
        [2],
        [2],
        [2],
        [3],
        [2],
        [3],
        [3],
        [2],
        [1],
        [1],
        [3],
        [1],
        [1],
        [3],
        [3],
        [2],
        [1],
        [1],
        [1],
        [2],
        [1],
        [1],
        [1],
        [1],
        [1],
        [3],
        [3],
        [2],
        [1],
        [2],
        [1],
        [3],
        [1],
        [2],
        [1],
        [3],
        [3],
        [1],
        [1],
        [1],
        [2],
        [1],
        [2],
        [0],
        [3],
        [1],
        [1],
        [1],
        [2],
        [2],
        [2],
        [3],
        [2],
        [1],
        [2],
        [2],
        [3],
        [2],
        [3],
        [1],
        [3],
        [1],
        [3],
        [3],
        [1],
        [2],
        [2],
        [3],
        [3],
        [3],
        [1],
        [1],
        [3],
        [3],
        [3],

In [22]:
onehot_labels_tr_tensor = one_hot_tensor(labels_tr_tensor, num_class)

In [23]:
# 将DataFrame的值转换为Tensor
labels_te_tensor = torch.LongTensor(labels_test.values)
labels_te_tensor

tensor([[2],
        [2],
        [1],
        [1],
        [2],
        [2],
        [3],
        [1],
        [3],
        [2],
        [2],
        [2],
        [2],
        [3],
        [2],
        [1],
        [1],
        [3],
        [3],
        [3],
        [1],
        [2],
        [3],
        [3],
        [1],
        [1],
        [3],
        [2],
        [3],
        [3],
        [1],
        [3],
        [3],
        [2],
        [1],
        [2],
        [3],
        [3],
        [2],
        [3],
        [1],
        [3],
        [2],
        [3],
        [3],
        [3],
        [2],
        [1],
        [1],
        [3],
        [1],
        [2],
        [3],
        [3],
        [1],
        [2],
        [1],
        [2],
        [1],
        [3],
        [2],
        [2],
        [1],
        [1],
        [3],
        [2],
        [2],
        [2],
        [1],
        [1],
        [3],
        [2],
        [2],
        [2],
        [1],
        [2],
        [1],

In [24]:
# 将DataFrame的值转换为Tensor
labels_tr_tensor = torch.LongTensor(labels_train.values)
labels_tr_tensor

tensor([[3],
        [1],
        [2],
        [2],
        [2],
        [3],
        [2],
        [3],
        [3],
        [2],
        [1],
        [1],
        [3],
        [1],
        [1],
        [3],
        [3],
        [2],
        [1],
        [1],
        [1],
        [2],
        [1],
        [1],
        [1],
        [1],
        [1],
        [3],
        [3],
        [2],
        [1],
        [2],
        [1],
        [3],
        [1],
        [2],
        [1],
        [3],
        [3],
        [1],
        [1],
        [1],
        [2],
        [1],
        [2],
        [0],
        [3],
        [1],
        [1],
        [1],
        [2],
        [2],
        [2],
        [3],
        [2],
        [1],
        [2],
        [2],
        [3],
        [2],
        [3],
        [1],
        [3],
        [1],
        [3],
        [3],
        [1],
        [2],
        [2],
        [3],
        [3],
        [3],
        [1],
        [1],
        [3],
        [3],
        [3],

In [25]:
# # 将DataFrame的值转换为Tensor
# labels_val_tensor = torch.LongTensor(labels_val.values)
# labels_val_tensor

In [26]:
# 定义数据集
train_dataset = TensorDataset(Methylation_train, miRNASeq_train, RNAseq_train, labels_tr_tensor)
test_dataset = TensorDataset(Methylation_test, miRNASeq_test, RNAseq_test, labels_te_tensor)
# val_dataset = TensorDataset(Methylation_val, miRNASeq_val, RNAseq_val, labels_val_tensor)

# 定义 DataLoader，使用自定义采样器
train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  # 测试集一般不需要打乱顺序
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  # 验证集也一般不需要重复采样

In [27]:
for i, batch in enumerate(train_loader):
    print(f"Batch {i + 1}: {len(batch[0])} samples")


Batch 1: 101 samples
Batch 2: 101 samples
Batch 3: 99 samples


In [28]:
for i, batch in enumerate(test_loader):
    print(f"Batch {i + 1}: {len(batch[0])} samples")

Batch 1: 100 samples


In [29]:
# for i, batch in enumerate(val_loader):
#     print(f"Batch {i + 1}: {len(batch[0])} samples")

In [30]:
import os
import matplotlib.pyplot as plt
from matplotlib import rcParams
from typing import Dict, Tuple, List
import gc

def plot_metrics(lr: float, depth: int, heads: int, dim_head: int, beta: float, adj_parameter: int, dropout: float) -> None:
    """生成四分类训练指标图表"""
    
    # ================== 配置参数 ==================
    SAVE_DIR = '../../result/BLCA_parting/'
    FILE_FORMAT = 'pdf'
    DPI = 600
    FONT_CONFIG = {
        'font.family': 'DejaVu Sans',
        'font.size': 12,
        'axes.titlesize': 14,
        'axes.labelsize': 12,
        'legend.fontsize': 10
    }
    STYLE_CONFIG = {
        'train': {'color': '#E74C3C', 'ls': '-', 'lw': 2.5, 'alpha': 0.9},
        'val': {'color': '#3498DB', 'ls': '-', 'lw': 2.5, 'alpha': 0.9},
        'test': {'color': '#2ECC71', 'ls': '-', 'lw': 2.5, 'alpha': 0.9}
    }
    
    # ================== 初始化设置 ==================
    rcParams.update(FONT_CONFIG)
    os.makedirs(SAVE_DIR, exist_ok=True)
    param_id = f"lr{lr:.0e}_depth{depth}heads{heads}dim_head{dim_head}adj_parameter{adj_parameter}dropout{dropout}beta{beta}".replace('.', '').replace('+', '')

    try:
        # ================== 创建画布 ==================
        fig = plt.figure(figsize=(18, 12), dpi=DPI, facecolor='white')
        fig.suptitle(f'Training Metrics ({param_id})', y=1.02, fontsize=16, fontweight='bold')
        
        # ================== 子图配置 ==================
        axes = [
            fig.add_subplot(221),  # Loss
            fig.add_subplot(222),  # Accuracy
            fig.add_subplot(223),  # F1 Weighted
            fig.add_subplot(224)   # F1 Macro
        ]
        
        metric_data = [
            ('Loss', (train_losses, test_losses), 'log' if max(train_losses) > 1e3 else 'linear'),
            ('Accuracy', (train_accuracies, test_accuracies), 'linear'),
            ('F1 Weighted', (train_f1_weighted_scores, test_f1_weighted_scores), 'linear'),
            ('F1 Macro', (train_f1_macro_scores, test_f1_macro_scores), 'linear')
        ]

        # ================== 绘制图表 ==================
        for ax, (title, (train, test), scale) in zip(axes, metric_data):
            ax.plot(train, label='Train', **STYLE_CONFIG['train'])
            # ax.plot(val, label='Val', **STYLE_CONFIG['val'])
            ax.plot(test, label='Test', **STYLE_CONFIG['test'])
            
            ax.set_title(title, pad=15, fontweight='semibold')
            ax.set_xlabel('Epochs', labelpad=10)
            ax.set_ylabel(title, labelpad=10)
            ax.grid(True, which='both', ls='--', alpha=0.4)
            ax.set_yscale(scale)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            
            if ax == axes[0]:
                handles, labels = ax.get_legend_handles_labels()
                fig.legend(handles, labels, loc='upper center', 
                         bbox_to_anchor=(0.5, 1.0), ncol=3, frameon=False)

        # ================== 布局优化 ==================
        plt.tight_layout(pad=3.0, w_pad=2.0, h_pad=2.0)
        
        # ================== 保存图像 ==================
        save_path = os.path.join(SAVE_DIR, f'metrics_{param_id}.{FILE_FORMAT}')
        plt.savefig(
            save_path,
            dpi=DPI,
            bbox_inches='tight',
            facecolor=fig.get_facecolor(),
            edgecolor='none',
            transparent=False,
            metadata={'CreationDate': None}
        )
        
    except Exception as e:
        print(f"图表生成失败: {str(e)}")
    finally:
        plt.close(fig)
        gc.collect()

In [31]:
def train_epoch(data_loader, model, optimizer, adj_parameter, output_attentions=False):
    model.train()
    total_loss = 0.0
    all_probs, all_labels = [], []

    for batch in data_loader:
        Methylation_batch, miRNASeq_batch, RNAseq_batch, labels_batch = [
            tensor.to(device) for tensor in batch
        ]

        # 生成三维邻接张量
        graph_train = np.stack([
            gen_trte_adj_mat(Methylation_batch.cpu(), adj_parameter),
            gen_trte_adj_mat(miRNASeq_batch.cpu(), adj_parameter),
            gen_trte_adj_mat(RNAseq_batch.cpu(), adj_parameter)
        ], axis=-1)
        graph_train = torch.from_numpy(graph_train).float().to(device).permute(0, 2, 1)

        # 梯度更新流程
        optimizer.zero_grad()
        ci_loss, ci = model(
            Methylation_batch, 
            miRNASeq_batch,
            RNAseq_batch,
            graph_train,
            labels_batch,
            output_attentions=output_attentions
        )
        ci_loss.backward()
        optimizer.step()

        # 结果收集
        total_loss += ci_loss.item() * labels_batch.size(0)
        all_probs.append(F.softmax(ci, dim=1).detach().cpu().numpy())
        all_labels.append(labels_batch.detach().cpu().numpy().flatten())

    return (
        total_loss / len(data_loader.dataset),
        np.concatenate(all_probs),
        np.concatenate(all_labels)
    )

In [32]:
# def val_epoch(data_loader, model, adj_parameter, output_attentions=False):
#     model.eval()
#     total_loss = 0.0
#     all_labels, all_probs = [], []

#     with torch.no_grad():
#         for batch in data_loader:
#             # 数据预处理
#             Methylation_batch, miRNASeq_batch, RNAseq_batch, labels_batch = [
#                 x.to(device) for x in batch
#             ]
#             # print(miRNASeq_batch.shape) #torch.Size([32, 217])
#             # 生成三维邻接张量
#             graph_val = torch.from_numpy(np.stack([
#                 gen_trte_adj_mat(d.cpu(), adj_parameter) 
#                 for d in [Methylation_batch, miRNASeq_batch, RNAseq_batch]
#             ], axis=-1)).float().to(device).permute(0,2,1)

#             # 前向计算
#             ci_loss, ci = model(
#                 Methylation_batch, 
#                 miRNASeq_batch,
#                 RNAseq_batch,
#                 graph_val,
#                 labels_batch,
#                 output_attentions=output_attentions
#             )

#             # 累计结果
#             total_loss += ci_loss.item() * labels_batch.size(0)
#             all_probs.append(F.softmax(ci, 1).detach().cpu().numpy())
#             all_labels.append(labels_batch.detach().cpu().numpy().flatten())

#     return (
#         total_loss / len(data_loader.dataset),
#         np.concatenate(all_probs),
#         np.concatenate(all_labels)
#     )

In [33]:
def test_epoch(data_loader, model, adj_parameter, output_attentions=False):
    model.eval()
    total_loss = 0.0
    all_probs, all_labels = [], []

    with torch.no_grad():
        for batch in data_loader:
            # 数据预处理 (保持原始变量名)
            Methylation_batch, miRNASeq_batch, RNAseq_batch, labels_test_tensor = [
                tensor.to(device) for tensor in batch
            ]

            # 生成邻接矩阵 (保持原始逻辑但更紧凑)
            graph_test = torch.from_numpy(
                np.stack([
                    gen_trte_adj_mat(Methylation_batch.cpu(), adj_parameter),
                    gen_trte_adj_mat(miRNASeq_batch.cpu(), adj_parameter),
                    gen_trte_adj_mat(RNAseq_batch.cpu(), adj_parameter)
                ], axis=-1)
            ).float().to(device).permute(0, 2, 1)

            # 前向计算
            loss, logits = model(
                Methylation_batch,
                miRNASeq_batch,
                RNAseq_batch,
                graph_test,
                labels_test_tensor,
                output_attentions=output_attentions
            )

            # 结果收集
            total_loss += loss.item() * labels_test_tensor.size(0)
            all_probs.append(F.softmax(logits, dim=1).detach().cpu().numpy())
            all_labels.append(labels_test_tensor.detach().cpu().numpy().flatten())

    return (
        total_loss / len(data_loader.dataset),
        np.concatenate(all_probs),
        np.concatenate(all_labels)
    )

In [34]:
# 用于保存指标数据的列表（移除了AUC相关指标）
train_losses = []
val_losses = []
test_losses = []
val_accuracies = []
val_f1_weighted_scores = []
val_f1_macro_scores = []
test_accuracies = []
test_f1_weighted_scores = []
test_f1_macro_scores = []
train_accuracies = []
train_f1_weighted_scores = []
train_f1_macro_scores = []

def train_test(
        Methylation_train,
        miRNASeq_train,
        RNAseq_train,
        lr,
        depth, 
        heads, 
        dim_head,
        beta, 
        attn_dropout,
        ff_dropout, 
        classifier_dropout,
        adj_parameter,
        num_epoch_pretrain
):
    # 初始化最佳指标跟踪器（以val_f1_macro为选择标准）
    best_metrics = {
        'epoch': -1,
        # 'val_f1_macro': -1.0,
        'test_acc': 0.0,
        'test_f1_weighted': 0.0,
        'test_f1_macro': 0.0,
    }
    
    DEPTH = depth
    HEAD = heads
    HEAD_DIM = dim_head

    classifier_input = [Methylation_train.shape[1], miRNASeq_train.shape[1], RNAseq_train.shape[1]]
    classifier_dim = [
        [classifier_input[0], 300, 200],
        [classifier_input[1], 300, 200],
        [classifier_input[2], 300, 200]
    ]
    col_dim = 32

    # 确定 row_dim 和 embeding_num
    if num_view <= 2:
        embeding = True
        embeding_num = 32
        row_dim = embeding_num
    else:
        embeding = False
        row_dim = num_view
        embeding_num = 32
    
    model = pathformer_model(
        row_dim=row_dim,
        col_dim=col_dim,
        depth=DEPTH,
        heads=HEAD,
        dim_head=HEAD_DIM,
        classifier_input=classifier_input,
        classifier_dim=classifier_dim,
        label_dim=label_dim,
        embeding=embeding,
        embeding_num=embeding_num,
        beta=beta,
        attn_dropout=attn_dropout,
        ff_dropout=ff_dropout,
        classifier_dropout=classifier_dropout,
        num_view=num_view,
        dim_hvcdn=dim_hvcdn
    )

    if torch.cuda.is_available():
        model.to(device)

    logging.info("\nPretrain GCNs...")
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    logging.info("\nTraining...")
    for epoch in range(num_epoch_pretrain + 1):
        # 训练阶段
        train_loss, train_probs, labels_train_tensor = train_epoch(train_loader, model, optimizer, adj_parameter, output_attentions=False)
        train_acc = accuracy_score(labels_train_tensor, train_probs.argmax(1))
        train_f1_weighted = f1_score(labels_train_tensor, train_probs.argmax(1), average='weighted')
        train_f1_macro = f1_score(labels_train_tensor, train_probs.argmax(1), average='macro')

        logging.info(f"Train Epoch {epoch}, Loss: {train_loss}")
        logging.info(f"Train Epoch {epoch}, ACC: {train_acc:.3f}")
        logging.info(f"Train Epoch {epoch}, F1 Weighted: {train_f1_weighted:.3f}")
        logging.info(f"Train Epoch {epoch}, F1 Macro: {train_f1_macro:.3f}")
        
        # 验证阶段
        # val_loss, val_probs, labels_validation_tensor = val_epoch(val_loader, model, adj_parameter, output_attentions=False)
        # val_acc = accuracy_score(labels_validation_tensor, val_probs.argmax(1))
        # val_f1_weighted = f1_score(labels_validation_tensor, val_probs.argmax(1), average='weighted')
        # val_f1_macro = f1_score(labels_validation_tensor, val_probs.argmax(1), average='macro')
        
        # logging.info(f"Val Epoch {epoch}, Loss: {val_loss}")
        # logging.info(f"Val Epoch {epoch}, ACC: {val_acc:.3f}")
        # logging.info(f"Val Epoch {epoch}, F1 Weighted: {val_f1_weighted:.3f}")
        # logging.info(f"Val Epoch {epoch}, F1 Macro: {val_f1_macro:.3f}")
        
        # 测试阶段
        test_loss, test_probs, labels_test_tensor = test_epoch(test_loader, model, adj_parameter, output_attentions=False)
        test_acc = accuracy_score(labels_test_tensor, test_probs.argmax(1))
        test_f1_weighted = f1_score(labels_test_tensor, test_probs.argmax(1), average='weighted')
        test_f1_macro = f1_score(labels_test_tensor, test_probs.argmax(1), average='macro')
        
        logging.info(f"Test Epoch {epoch}, Loss: {test_loss}")
        logging.info(f"Test Epoch {epoch}, ACC: {test_acc:.3f}")
        logging.info(f"Test Epoch {epoch}, F1 Weighted: {test_f1_weighted:.3f}")
        logging.info(f"Test Epoch {epoch}, F1 Macro: {test_f1_macro:.3f}")

        # 更新最佳指标（以val_f1_macro为选择标准）
        if test_f1_weighted > best_metrics['test_f1_weighted'] or \
           (test_f1_weighted == best_metrics['test_f1_weighted'] and test_f1_macro > best_metrics['test_f1_macro']):
            best_metrics.update({
                'epoch': epoch,
                # 'val_f1_macro': val_f1_macro,
                'test_acc': test_acc,
                'test_f1_weighted': test_f1_weighted,
                'test_f1_macro': test_f1_macro
            })

        logging.info("\n")

        # 保存训练指标
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        train_f1_weighted_scores.append(train_f1_weighted)
        train_f1_macro_scores.append(train_f1_macro)
        
        # val_losses.append(val_loss)
        # val_accuracies.append(val_acc)
        # val_f1_weighted_scores.append(val_f1_weighted)
        # val_f1_macro_scores.append(val_f1_macro)
        
        test_losses.append(test_loss)
        test_accuracies.append(test_acc)
        test_f1_weighted_scores.append(test_f1_weighted)
        test_f1_macro_scores.append(test_f1_macro)

        # 绘制并保存指标图（需调整绘图函数）
        plot_metrics(lr, depth, heads, dim_head, beta, adj_parameter, dropout=attn_dropout)

    return (
        best_metrics['test_acc'],
        best_metrics['test_f1_weighted'],
        best_metrics['test_f1_macro']
    )

In [35]:
# # 用于保存指标数据的列表
# train_losses = []
# val_losses = []
# test_losses = []
# val_accuracies = []
# val_f1_scores = []
# val_auc_scores = []
# test_accuracies = []
# test_f1_scores = []
# test_auc_scores = []
# train_accuracies = []
# train_f1_scores = []
# train_auc_scores = []
# train_c_index = []
# val_c_index = []
# test_c_index = []

# def train_test(
#         Methylation_train,
#         Methylation_test,
#         miRNASeq_train,
#         miRNASeq_test,
#         RNAseq_train,
#         RNAseq_test,
#         label_train,
#         label_test, 
#         labels_tr_tensor,
#         lr,
#         depth, 
#         heads, 
#         dim_head,
#         beta, 
#         attn_dropout,
#         ff_dropout, 
#         classifier_dropout,
#         adj_parameter,
#         num_epoch_pretrain
# ):
    
#     DEPTH = depth
#     HEAD = heads
#     HEAD_DIM = dim_head

#     classifier_input = [Methylation_train.shape[1], miRNASeq_train.shape[1], RNAseq_train.shape[1]]
#     classifier_dim = [
#         [classifier_input[0], 300, 200],
#         [classifier_input[1], 300, 200],
#         [classifier_input[2], 300, 200]
#     ]
#     col_dim = 71

#     # 确定 row_dim 和 embeding_num
#     if num_view <= 2:
#         embeding = True
#         embeding_num = 71
#         row_dim = embeding_num  # 请根据需要修改此处
#     else:
#         embeding = False
#         row_dim = num_view
#         embeding_num = 71
    
#     model = pathformer_model(
#         row_dim=row_dim,
#         col_dim=col_dim,
#         depth=DEPTH,
#         heads=HEAD,
#         dim_head=HEAD_DIM,
#         classifier_input=classifier_input,
#         classifier_dim=classifier_dim,
#         label_dim=label_dim,
#         embeding=embeding,
#         embeding_num=embeding_num,
#         beta=beta,
#         attn_dropout=attn_dropout,
#         ff_dropout=ff_dropout,
#         classifier_dropout=classifier_dropout,
#         num_view=num_view,
#         dim_hvcdn=dim_hvcdn
#     )

#     if torch.cuda.is_available():
#         model.to(device)

#     print("\nPretrain GCNs...")
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#     print("\nTraining...")
#     for epoch in range(num_epoch_pretrain + 1):
#         # 每一轮都进行训练
#         train_loss, train_probs, labels_train_tensor = train_epoch(train_loader, model, optimizer, adj_parameter, epoch, output_attentions=False)
#         train_acc = accuracy_score(labels_train_tensor, train_probs.argmax(1))
#         train_f1 = f1_score(labels_train_tensor, train_probs.argmax(1))
#         train_auc = roc_auc_score(labels_train_tensor, train_probs[:, 1])
# #         print(f"Train Epoch {epoch}, Loss: {train_loss}")
# #         print(f"Train Epoch {epoch}, ACC: Train ACC: {train_acc:.3f}")
# #         print(f"Train Epoch {epoch}, f1: Train F1: {train_f1:.3f}")
# #         print(f"Train Epoch {epoch}, AUC: Train AUC: {train_auc:.3f}")
        
#         # 每一轮都进行验证
#         val_loss, val_probs, labels_val_tensor = val_epoch(val_loader, model, adj_parameter, epoch, output_attentions=False)
#         val_acc = accuracy_score(labels_val_tensor, val_probs.argmax(1))
#         val_f1 = f1_score(labels_val_tensor, val_probs.argmax(1))
#         val_auc = roc_auc_score(labels_val_tensor, val_probs[:, 1])
# #         print(f"val Epoch {epoch}, Loss: {val_loss}")
# #         print(f"val Epoch {epoch}, ACC: val ACC: {val_acc:.3f}")
# #         print(f"val Epoch {epoch}, f1: val F1: {val_f1:.3f}")
# #         print(f"val Epoch {epoch}, AUC: val AUC: {val_auc:.3f}")
        
#         # 每一轮都进行测试
#         test_loss, test_probs, labels_test_tensor = test_epoch(test_loader, model, adj_parameter, epoch, output_attentions=False)
#         test_acc = accuracy_score(labels_test_tensor, test_probs.argmax(1))
#         test_f1 = f1_score(labels_test_tensor, test_probs.argmax(1))
#         test_auc = roc_auc_score(labels_test_tensor, test_probs[:, 1])
# #         print(f"test Epoch {epoch}, Loss: {test_loss}")
# #         print(f"test Epoch {epoch}, ACC: test ACC: {test_acc:.3f}")
# #         print(f"test Epoch {epoch}, f1: test F1: {test_f1:.3f}")
# #         print(f"test Epoch {epoch}, AUC: test AUC: {test_auc:.3f}")

#         print("\n")

#         # 保存训练指标
#         train_losses.append(train_loss)
#         train_accuracies.append(train_acc)
#         train_f1_scores.append(train_f1)
#         train_auc_scores.append(train_auc)
        
#         val_losses.append(val_loss)
#         val_accuracies.append(val_acc)
#         val_f1_scores.append(val_f1)
#         val_auc_scores.append(val_auc)
        
#         test_losses.append(test_loss)
#         test_accuracies.append(test_acc)
#         test_f1_scores.append(test_f1)
#         test_auc_scores.append(test_auc)

#         # 绘制并保存指标图
#         plot_metrics(lr, depth, heads, dim_head, beta, dropout=attn_dropout)

#     # 返回最终的预测结果和指标
#     final_predictions = test_probs.argmax(1)
#     return final_predictions, test_acc, test_f1, test_auc


In [36]:
import logging
import os
import time
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# 配置日志和路径
LOG_FILE = '../../result/BLCA_parting/training_log_CMFM.txt'
RESULT_FILE = '../../result/BLCA_parting/mogat_training_results.xlsx'
os.makedirs(os.path.dirname(LOG_FILE), exist_ok=True)

logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    filemode='w'  # 每次运行覆盖旧日志
)

# 超参数配置（便于维护和修改）
HP_CONFIG = {
    'lr': [0.0001, 0.001, 0.01],
    'depth': [2, 3, 4, 6],
    'heads': [4, 6, 8],
    'dim_head': [12, 16, 32, 64],
    'dropout': [0.3],
    'beta': [1],
    'adj_parameter': [2, 4, 6, 8, 10]
}

# 计算总组合数
total_combinations = (
    len(HP_CONFIG['lr']) *
    len(HP_CONFIG['depth']) *
    len(HP_CONFIG['heads']) *
    len(HP_CONFIG['dim_head']) *
    len(HP_CONFIG['dropout']) *
    len(HP_CONFIG['beta']) *
    len(HP_CONFIG['adj_parameter'])
)

# 结果缓存配置
BATCH_SIZE = 5  # 每5个结果写入一次文件
results_cache = []

def save_results(batch: list, force_save: bool = False):
    """批量保存结果到文件"""
    if not force_save and len(batch) < BATCH_SIZE:
        return
    
    try:
        df = pd.DataFrame(batch)
        header = not os.path.exists(RESULT_FILE)
        
        with pd.ExcelWriter(
            RESULT_FILE,
            mode='a' if os.path.exists(RESULT_FILE) else 'w',
            engine='openpyxl',
            if_sheet_exists='overlay' if os.path.exists(RESULT_FILE) else None
        ) as writer:
            df.to_excel(
                writer,
                index=False,
                sheet_name='Results',
                header=header,
                startrow=writer.sheets['Results'].max_row if not header else 0
            )
        batch.clear()
    except Exception as e:
        logging.error(f"保存结果失败: {str(e)}")
        raise

# 主训练循环
current_combination = 0
start_total = time.time()

try:
    for lr in HP_CONFIG['lr']:
        for depth in HP_CONFIG['depth']:
            for heads in HP_CONFIG['heads']:
                for dim_head in HP_CONFIG['dim_head']:
                    for dropout in HP_CONFIG['dropout']:
                        for beta in HP_CONFIG['beta']:
                            for adj_param in HP_CONFIG['adj_parameter']:
                                current_combination += 1
                                start_iter = time.time()
                                
                                # 日志记录当前参数
                                param_str = (
                                    f"lr={lr}, depth={depth}, heads={heads}, "
                                    f"dim_head={dim_head}, dropout={dropout}, "
                                    f"beta={beta}, adj_parameter={adj_param} "
                                    f"({current_combination}/{total_combinations})"
                                )
                                logging.info(f"开始训练: {param_str}")

                                try:
                                    # 执行训练（假设train_test返回acc, f1_weighted, f1_macro）
                                    acc, f1_weighted, f1_macro = train_test(  # 修改变量接收
                                        Methylation_train=Methylation_train,
                                        miRNASeq_train=miRNASeq_train,
                                        RNAseq_train=RNAseq_train,
                                        lr=lr,
                                        depth=depth,
                                        heads=heads,
                                        dim_head=dim_head,
                                        beta=beta,
                                        attn_dropout=dropout,
                                        ff_dropout=dropout,
                                        classifier_dropout=dropout,
                                        adj_parameter=adj_param,
                                        num_epoch_pretrain=1500
                                    )

                                    # 缓存结果（修改键名）
                                    results_cache.append({
                                        'lr': lr,
                                        'depth': depth,
                                        'heads': heads,
                                        'dim_head': dim_head,
                                        'dropout': dropout,
                                        'beta': beta,
                                        'adj_parameter': adj_param,
                                        'accuracy': acc,
                                        'f1_weighted': f1_weighted,  # 修改键名
                                        'f1_macro': f1_macro          # 新增键
                                    })

                                    # 定期保存
                                    save_results(results_cache)

                                    # 记录成功信息（修改日志输出）
                                    elapsed = time.time() - start_iter
                                    logging.info(
                                        f"完成: {param_str} | "
                                        f"耗时: {elapsed:.1f}s | "
                                        f"结果: ACC={acc:.4f}, F1_weighted={f1_weighted:.4f}, F1_macro={f1_macro:.4f}"  # 修改指标名称
                                    )

                                except torch.cuda.OutOfMemoryError:
                                    logging.warning(f"显存不足，跳过组合: {param_str}")
                                    torch.cuda.empty_cache()
                                except Exception as e:
                                    logging.error(f"训练失败: {param_str} | 错误: {str(e)}")
                                    continue
                                # 用于保存指标数据的列表（移除了AUC相关指标）
                                train_losses = []
                                val_losses = []
                                test_losses = []
                                val_accuracies = []
                                val_f1_weighted_scores = []
                                val_f1_macro_scores = []
                                test_accuracies = []
                                test_f1_weighted_scores = []
                                test_f1_macro_scores = []
                                train_accuracies = []
                                train_f1_weighted_scores = []
                                train_f1_macro_scores = []

    # 最后强制保存剩余结果
    save_results(results_cache, force_save=True)

except KeyboardInterrupt:
    logging.warning("用户中断训练，保存已完成的实验结果...")
    save_results(results_cache, force_save=True)
finally:
    total_time = time.time() - start_total
    logging.info(f"全部完成! 总耗时: {total_time/3600:.2f} 小时")

print("实验完成，结果保存在:", RESULT_FILE)

In [ ]:
# import os
# import time
# import pandas as pd
# from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# # 创建一个 DataFrame 用于存储结果
# results_df = pd.DataFrame(columns=['lr', 'depth', 'heads', 'dim_head', 'dropout', 'beta', 'accuracy', 'f1_score', 'auc'])

# # 设置单一的超参数组合
# lr = 0.0001  # 你想要的学习率
# depth = 6  # 你想要的层数
# heads = 4  # 你想要的注意力头数
# dim_head = 32  # 你想要的注意力头的维度
# dropout = 0.5  # 你想要的 dropout 比例
# beta = 1  # 你想要的 beta 值

# # 训练参数组合
# start_time = time.time()  # 记录开始时间
# print(f"训练参数：lr={lr}, depth={depth}, heads={heads}, "
#       f"dim_head={dim_head}, dropout={dropout}, beta={beta}")

# # 训练函数假设已定义，返回最后的预测和各项指标
# try:
#     last_predictions, acc, f1, auc = train_test(
#         Methylation_train=Methylation_train,
#         Methylation_test=Methylation_test,
#         miRNASeq_train=miRNASeq_train,
#         miRNASeq_test=miRNASeq_test,
#         RNAseq_train=RNAseq_train,
#         RNAseq_test=RNAseq_test,
#         label_train=labels_train,
#         label_test=labels_test,
#         labels_tr_tensor=labels_tr_tensor,
#         lr=lr,
#         depth=depth,
#         heads=heads,
#         dim_head=dim_head,
#         beta=beta,
#         attn_dropout=dropout,
#         ff_dropout=dropout,
#         classifier_dropout=dropout,
#         num_epoch_pretrain=2000,
#     )

#     # 清空列表以便下一个超参数组合使用
#     train_losses = []
#     val_losses = []
#     test_losses = []
    
#     train_accuracies = []
#     val_accuracies = []
#     test_accuracies = []
    
#     train_f1_scores = []
#     val_f1_scores = []
#     test_f1_scores = []
    
#     train_auc_scores = []
#     val_auc_scores = []
#     test_auc_scores = []
    
#     # 检查最后的预测结果是否全部一致
#     if last_predictions is not None:
#         last_predictions_tensor = torch.tensor(last_predictions)  # 转换为张量
#         if not torch.all(last_predictions_tensor == last_predictions_tensor[0]):
#             print("最后一轮的预测结果不一致，保存该参数组合和评价指标。")
            
#             # 保存结果到 DataFrame
#             result = {
#                 'lr': lr,
#                 'depth': depth,
#                 'heads': heads,
#                 'dim_head': dim_head,
#                 'dropout': dropout,
#                 'beta': beta,
#                 'accuracy': acc,
#                 'f1_score': f1,
#                 'auc': auc
#             }

#             result_df = pd.DataFrame([result])

#             # 确保 result_df 不为空且不全为 NA
#             if result_df.notnull().any().any():
#                 results_df = pd.concat([results_df, result_df], ignore_index=True)

#                 # 检查 Excel 文件是否存在，如果不存在则创建
#                 if not os.path.isfile('../../result/BLCA_parting/mogat_training_results.xlsx'):
#                     results_df.to_excel('../../result/BLCA_parting/mogat_training_results.xlsx', index=False, sheet_name='Results')
#                 else:
#                     # 立即写入 Excel 文件
#                     with pd.ExcelWriter('../../result/BLCA_parting/mogat_training_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
#                         results_df.to_excel(writer, index=False, sheet_name='Results', startrow=writer.sheets['Results'].max_row)
#         else:
#             print("最后一轮的预测结果全部一致，不保存该参数组合。")

# except Exception as e:
#     print(f"训练参数组合 lr={lr}, depth={depth}, heads={heads}, dim_head={dim_head}, dropout={dropout}, beta={beta} 时发生错误: {e}")

# elapsed_time = time.time() - start_time
# print(f"训练完成，耗时: {elapsed_time:.2f} 秒")

# # 最终保存（如果需要）
# if not results_df.empty:
#     results_df.to_excel('../../result/BLCA_parting/mogat_training_results.xlsx', index=False, sheet_name='Results')
#     print("参数组合和评价指标已保存为 mogat_training_results.xlsx 文件。")
# else:
#     print("没有符合条件的参数组合，未保存任何结果。")
